## Making Trained Model table more flexible

In [1]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True
dj.config['schema_name'] = "nnfabrik_example"

from nnfabrik.main import *
from nnfabrik.templates.transfer import TransferredTrainedModelBase
from nnfabrik.templates.transfer.recipes import DatasetTransferRecipe, ModelTransferRecipe, TrainerTransferRecipe

Connecting anix@134.2.168.16:3306


In [2]:
schema.drop()

Proceed to delete entire schema `nnfabrik_example`? [yes, No]:  yes


In [2]:
@schema
class TransferredTrainedModel(TransferredTrainedModelBase):
    table_comment = "Transferred trained models"

## Add some entries to the table (usual step)

In [3]:
fabrikant_info = dict(fabrikant_name="Your Name", email="your@email.com", affiliation="thelab", dj_username="yourname")
Fabrikant().insert1(fabrikant_info)

In [4]:
Seed().insert([{'seed':7}])

In [5]:
dataset_fn = "nnfabrik.examples.mnist.dataset.mnist_dataset_fn"

dataset_config = dict(batch_size=64) # we specify all the inputs except the ones required by nnfabrik

Dataset().add_entry(dataset_fn=dataset_fn, dataset_config=dataset_config, 
                    dataset_fabrikant="Your Name", dataset_comment="A comment about the dataset!");

In [6]:
# specify model function as string (the function must be importable) as well as the model config
model_fn = "nnfabrik.examples.mnist.model.mnist_model_fn"
model_config = dict(h_dim=5) # we specify all the inputs except the ones required by nnfabrik

Model().add_entry(model_fn=model_fn, model_config=model_config, 
                  model_fabrikant="Your Name", model_comment="A comment about the model!");

In [7]:
# specify trainer function as string (the function must be importable) as well as the trainer config
trainer_fn = "nnfabrik.examples.mnist.trainer.mnist_trainer_fn"
trainer_config = dict(epochs=1) # we specify all the inputs except the ones required by nnfabrik

Trainer().add_entry(trainer_fn=trainer_fn, trainer_config=trainer_config, 
                  trainer_fabrikant="Your Name", trainer_comment="A comment about the trainer!");

## Populating TrainedModel normally

In [11]:
TransferredTrainedModel.populate()

In [12]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,7,,,,,,,A comment about the trainer!.A comment about the model!.A comment about the dataset!,72.3317,=BLOB=,None,2020-10-27 16:12:19,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0


Below are the possible cases where one might want to transfer something from an existing set of trained models to a new set of trained models:
* you trained models with a specific trainer and you want to continue training them with a new trainer
* you trained models with a specific dataset and you want to 

This boils down to:
* what datasets and/or models and/or trainers you would like to use at a specific transfer step -> specify this in the recipe
* which already-exisiting entries you would like this transfer step to be applied on -> specify this in the `populate` restrictions

## Adding recipes after populating TrainedModel

First, let's add the model that we want to use for the transfer. 

In [14]:
dataset_fn = "nnfabrik.examples.mnist.dataset.mnist_dataset_fn"

dataset_config = dict(batch_size=64, apply_augmentation=True) # we specify all the inputs except the ones required by nnfabrik

Dataset().add_entry(dataset_fn=dataset_fn, dataset_config=dataset_config, 
                    dataset_fabrikant="Your Name", dataset_comment="Augmented MNIST");

In [15]:
Dataset()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,dataset_config dataset configuration object,dataset_fabrikant Name of the contributor that added this entry,dataset_comment short description,dataset_ts UTZ timestamp at time of insertion
nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,=BLOB=,Your Name,Augmented MNIST,2020-10-27 16:22:54
nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,=BLOB=,Your Name,A comment about the dataset!,2020-10-27 16:11:53


#### Define the recipe

In [17]:
transfer_from = {"dataset_fn": 'nnfabrik.examples.mnist.dataset.mnist_dataset_fn', "dataset_hash": '9aee736870714f8b7c3cc084087ce886'}
transfer_to = {"dataset_fn": 'nnfabrik.examples.mnist.dataset.mnist_dataset_fn', "dataset_hash": '28aefc2308569727c6017c66c9122d77'}

In [19]:
DatasetTransferRecipe().add_entry(transfer_from=transfer_from, transfer_to=transfer_to, transfer_step=1)

In [20]:
DatasetTransferRecipe()

transfer_step,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,prev_dataset_fn name of the dataset loader function,prev_dataset_hash hash of the configuration object
1,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886


### Add the recipes to the table

In [21]:
TransferredTrainedModel.transfer_recipe = [DatasetTransferRecipe()]

### Populate the table for transfer step 1

In [22]:
TransferredTrainedModel.populate()

100%|██████████| 938/938 [00:19<00:00, 47.43it/s]


In [23]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,7,,,,,,,A comment about the trainer!.A comment about the model!.A comment about the dataset!,72.3317,=BLOB=,None,2020-10-27 16:12:19,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0
1,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,7,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,A comment about the trainer!.A comment about the model!.Augmented MNIST,72.3317,=BLOB=,None,2020-10-27 16:27:41,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0


### Populate the table for transfer step 2

In [26]:
TransferredTrainedModel.populate()

IndexError: list index out of range

In [32]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,,,,,,,professional trainer.SOA Model.beautiful dataset,0.15854,=BLOB=,Mohammad Bashiri,2020-04-06 15:12:50,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,7,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,-0.00758789,=BLOB=,Mohammad Bashiri,2020-04-06 15:13:03,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,0.0218181,=BLOB=,Mohammad Bashiri,2020-04-06 15:13:16,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84
2,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,7,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,-0.00751,=BLOB=,Mohammad Bashiri,2020-04-06 15:13:28,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd
2,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,7,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.r